In [2]:
import re
import sys
import json
import tarfile
import urllib3
import configparser

import tensorflow_hub as hub
import tensorflow.compat.v1 as tf

from tqdm import tqdm
from gensim.models import Doc2Vec
from elasticsearch import Elasticsearch

In [3]:
urllib3.disable_warnings()

config = configparser.ConfigParser()
config.read('../server.ini');

In [4]:
es = Elasticsearch(
    config['ELASTIC']['path'],
    verify_certs=False
)

/opt/homebrew/Caskroom/miniconda/base/envs/thesis/lib/python3.10/site-packages/elasticsearch/connection/http_urllib3.py:209: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [5]:
es.info()

{'name': '1bb166804bb3',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'm81m9H08RKigCdwIje8C6w',
 'version': {'number': '8.11.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'd9ec3fa628c7b0ba3d25692e277ba26814820b20',
  'build_date': '2023-11-04T10:04:57.184859352Z',
  'build_snapshot': False,
  'lucene_version': '9.8.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [6]:
model = Doc2Vec.load('../out/models/word2vec.model')

In [7]:
specs = []
specs_names = []
specs_versions = []

tar = tarfile.open('../data/data.tar.gz', 'r:gz')
nl_tags = r'(?:\'|\")(?:description|name|title|summary)(?:\'|\"):\s(?:\'|\")([^\'\"]+)(?:\'|\")'

for spec in tqdm(tar.getmembers(), file=sys.stdout):
    file = tar.extractfile(spec)
    
    if file is not None:
        try:
            contents = json.load(file)
            
            specs_names.append(contents['info']['title'])
            specs_versions.append(contents['info']['version'] if not contents['info']['version'].startswith('v') else contents['info']['version'][1:])
            specs.append(' '.join(re.findall(nl_tags, str(contents))))
        except json.JSONDecodeError:
            continue

 50%|████▉     | 1983/3996 [00:05<00:05, 390.76it/s]


KeyboardInterrupt: 

In [8]:
print('Downloading pre-trained embeddings from tensorflow hub...')
tf.disable_eager_execution()
embed = hub.Module('https://tfhub.dev/google/universal-sentence-encoder/2')
text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)
print('Done.')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Done.


In [9]:
print('Creating tensorflow session...')
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
print('Done.')

Creating tensorflow session...


2023-11-13 02:05:46.255951: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-13 02:05:47.160057: W tensorflow/c/c_api.cc:305] Operation '{name:'module/string_to_index_11/hash_table/table_init' id:3918 op device:{requested: '', assigned: ''} def:{{{node module/string_to_index_11/hash_table/table_init}} = InitializeTableV2[Tkey=DT_STRING, Tval=DT_INT64, _has_manual_control_dependencies=true](module/string_to_index_11/hash_table, module/Embeddings_en_words, module/string_to_index_11/ToInt64)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Done.


In [ ]:
es.indices.create(
    index='oa-specifications',
    body={
        'mappings': {
            'properties': {
                "document_vector": {
                    "type": "dense_vector",
                    "dims": 512
                },              
                "name" : {
                    "type" : "keyword"
                },
                "version" : {
                    "type" : "keyword"
                }
            }
        }
    }
)

for ind in tqdm(range(len(specs))):
    es.index(
        index='oa-specifications',
        body={
            'name': specs_names[ind],
            'version': specs_versions[ind],
            'document_vector': session.run(embeddings, feed_dict={text_ph: [specs[ind]]}).tolist()[0]
        }
    )

In [44]:
query = 'weather forecast by city'

knn_query = {
    'knn': {
        'field': 'document_vector',
        'query_vector': session.run(embeddings, feed_dict={text_ph: [query]}).tolist()[0],
        'k': 5,
        'num_candidates': 3992
    }
}

results = es.search(index='oa-specifications', body=knn_query, _source=['name', 'version'])['hits']['hits']

In [45]:
print(f'These are the top 10 results of the query "{query}":\n')

for ind, result in enumerate(results):
    length_name = max([len(el['_source']['name']) for el in results]) + 2
    length_version = max([len(el['_source']['version']) for el in results]) + 2

    print(f'\t{str(ind + 1) + ".": <2} {result["_source"]["name"]: <{length_name}} v.{result["_source"]["version"]: <{length_version}} [{str(result["_score"] * 100)[:2]}%]')

These are the top 10 results of the query "weather forecast by city":

	1. Storm Glass Marine Weather              v.1.0.1   [79%]
	2. Interzoid Get Weather City API          v.1.0.0   [78%]
	3. ODWeather                               v.1.0     [75%]
	4. Visual Crossing Weather API             v.4.6     [75%]
	5. Interzoid Get Weather By Zip Code API   v.1.0.0   [75%]
